## Observations and Insights 

Look across all previously generated figures and tables and write at least three observations or inferences that can be made from the data. Include these observations at the top of notebook.

Observation 1
Observation 2
Observation 3


In [ ]:
%matplotlib widget

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
scc_combined_df = pd.DataFrame.merge(mouse_metadata,study_results,on="Mouse ID",how="outer")

# Display the data table for preview
#scc_data_df.info()
scc_combined_df

In [ ]:
# Checking the number of mice.
scc_combined_df["Mouse ID"].nunique()

In [ ]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 

In [ ]:
# Optional: Get all the data for the duplicate mouse ID. 
duplicate_mice = scc_combined_df[scc_combined_df.duplicated(["Mouse ID","Timepoint"])]
duplicate_mice

In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
scc_cleaned_df = scc_combined_df.drop_duplicates(["Mouse ID","Timepoint"])
scc_cleaned_df

In [ ]:
# Checking the number of mice in the clean DataFrame.
num_mice = scc_cleaned_df["Mouse ID"].nunique()
num_mice

## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# This method is the most straightforward, creating multiple series and putting them all together at the end.

# Create a list of regimens
drug_regimen_list = scc_cleaned_df["Drug Regimen"].unique()
# Create variables to hold stats for each drug regimen
drug_list = []
mean_list = []
median_list = []
variance_list = []
stdev_list = []
sem_list = []

# Use a loop to create a series of tumor volume for each drug regimen
for drug in drug_regimen_list:
        drug_list.append(drug)
        mean_list.append(scc_cleaned_df.loc[(scc_cleaned_df["Drug Regimen"]==drug)]["Tumor Volume (mm3)"].mean())
        median_list.append(scc_cleaned_df.loc[(scc_cleaned_df["Drug Regimen"]==drug)]["Tumor Volume (mm3)"].median())
        variance_list.append(scc_cleaned_df.loc[(scc_cleaned_df["Drug Regimen"]==drug)]["Tumor Volume (mm3)"].var())
        stdev_list.append(scc_cleaned_df.loc[(scc_cleaned_df["Drug Regimen"]==drug)]["Tumor Volume (mm3)"].std())
        sem_list.append(scc_cleaned_df.loc[(scc_cleaned_df["Drug Regimen"]==drug)]["Tumor Volume (mm3)"].sem())
# Put lists together into a data frame
summary_stats_df = pd.DataFrame({"Drug Regimen" : drug_list,"mean" : mean_list, "median" : median_list, "var": variance_list, "std" : stdev_list, "sem" : sem_list}).sort_values("Drug Regimen").round(2).set_index("Drug Regimen")
summary_stats_df

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# This method produces everything in a single groupby function
scc_cleaned_df.groupby(["Drug Regimen"]).agg({"Tumor Volume (mm3)": ["mean","median","var","std","sem"]}).round(2)

## Bar and Pie Charts

In [ ]:
# Check mouse counts to verify bar plot values
drug_mouse_count = scc_cleaned_df.groupby(["Drug Regimen"],as_index=False).agg({"Mouse ID": pd.Series.nunique})
drug_mouse_count

In [ ]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 
# Group data by drug regimen and aggregate by number of mice (count)
# mice_by_regimen_df = scc_cleaned_df.groupby(["Drug Regimen"],as_index=False).agg({"Mouse ID": "count"})
# Plot data as a bar chart
bar_plot = drug_mouse_count.plot(kind="bar",x="Drug Regimen",y="Mouse ID",legend=False)
bar_plot.set_xlabel("Drug Regimen") # x axis labels
bar_plot.set_ylabel("Number of mice") # y axis labels
plt.tight_layout() # Fit plot to space

In [ ]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.
fig1,ax1 = plt.subplots()
drug_list = drug_mouse_count["Drug Regimen"].unique() # Add categories for x axis
x_axis = np.arange(0, len(drug_list)) # x axis length
num_mice = drug_mouse_count["Mouse ID"] # y axis values
tick_locations = [value for value in x_axis] # Add tick locations
plt.xticks(tick_locations,drug_list,rotation=90) # Add x tick labels
plt.xlabel("Drug Regimen") # x axis labels
plt.ylabel("Number of mice") # y axis labels
plt.tight_layout() # Fit plot to space
plt.bar(x_axis,num_mice) # Generate plot

In [ ]:
# Check mouse by sex to verify values in pit plots
sex_mouse_count = scc_cleaned_df.groupby(["Sex"],as_index=False).agg({"Mouse ID": pd.Series.nunique})
sex_mouse_count = sex_mouse_count.rename(columns={"Mouse ID":"Number of mice"})
sex_mouse_count

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas
pie_plot = sex_mouse_count.plot(kind="pie",y="Number of mice", labels=sex_mouse_count["Sex"],autopct='%1.1f%%',startangle=135,legend=False)
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
labels = sex_mouse_count["Sex"]
sizes = sex_mouse_count["Number of mice"]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=135)
plt.ylabel("Number of mice")
plt.show()

## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Create data frames for each relevant drug regimen
capomulin_df = scc_cleaned_df.loc[(scc_cleaned_df["Drug Regimen"]=="Capomulin")]
ramicane_df = scc_cleaned_df.loc[(scc_cleaned_df["Drug Regimen"]=="Ramicane")]
infubinol_df = scc_cleaned_df.loc[(scc_cleaned_df["Drug Regimen"]=="Infubinol")]
ceftamin_df =  scc_cleaned_df.loc[(scc_cleaned_df["Drug Regimen"]=="Ceftamin")]
# Concatenate individual drug regimen data frames
treatment_df = pd.concat([capomulin_df,ramicane_df,infubinol_df,ceftamin_df],ignore_index=True)
# treatment_df
# Start by getting the last (greatest) timepoint for each mouse
last_timepoint_df = treatment_df.groupby(["Mouse ID"],as_index=False).agg({"Timepoint" : pd.Series.max})
# Merge this group df with the original dataframe to get the tumor volume at the last timepoint
merged_df = pd.merge(last_timepoint_df,treatment_df,on="Mouse ID",how="left")
# Keep rows with final timepoint only
merged_df2 = merged_df.loc[merged_df["Timepoint_y"]==merged_df["Timepoint_x"]]
# Create new data frame with relvant data only
final_tumor_df = merged_df2[["Mouse ID","Timepoint_x","Drug Regimen","Sex","Age_months","Weight (g)","Tumor Volume (mm3)","Metastatic Sites"]]
# Rename column
final_tumor_df = final_tumor_df.rename(columns={"Timepoint_x":"Timepoint"})
# Reset index
final_tumor_df.reset_index(drop=True,inplace=True)
final_tumor_df.head()

In [ ]:
# Put treatments into a list for for loop (and later for plot labels)
treatment_list = final_tumor_df["Drug Regimen"].unique()
# treatment_list
# Create empty list to fill with tumor vol data (for plotting)
tumor_vol_data = []

# Calculate the IQR and quantitatively determine if there are any potential outliers. 
for treatment in treatment_list:
        quartiles = final_tumor_df.loc[(final_tumor_df["Drug Regimen"]==treatment)]["Tumor Volume (mm3)"].quantile([.25,.5,.75])
        lowerq = quartiles[0.25]
        upperq = quartiles[0.75]
        iqr = round(upperq-lowerq,2)
        lower_bound = round(lowerq - (1.5*iqr),2)
        upper_bound = round(upperq + (1.5*iqr),2)
        # Locate the rows which contain mice on each drug and get the tumor volumes
        tumor_vol_data.append(final_tumor_df.loc[(final_tumor_df["Drug Regimen"]==treatment)]["Tumor Volume (mm3)"])
        # add subset 
        # ???
        # Determine outliers using upper and lower bounds
        print(f"The interquartile range for {treatment} is {iqr}")
        print(f"Values below {lower_bound} could be outliers.")
        print(f"Values above {upper_bound} could be outliers.")

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest
fig1, ax1 = plt.subplots()
ax1.set_title("Tumor volume by drug")
ax1.set_ylabel("Tumor volume (mm3)")
ax1.set_xlabel("Drug Regimen")
green_diamond = dict(markerfacecolor='g', marker='D') # Change outlier marker
ax1.boxplot(tumor_vol_data, flierprops=green_diamond) # Plot with new outlier marker
plt.show()
plt.xticks([1,2,3,4],treatment_list)

## Line and Scatter Plots

In [ ]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin
# Use data frame created earlier:
# capomulin_df = scc_cleaned_df.loc[(scc_cleaned_df["Drug Regimen"]=="Capomulin")]
# capomulin_df
# Select one mouse
# capomulin_df["Mouse ID"].unique()
# Seleted mouse ID s185
s185_df = capomulin_df.loc[(capomulin_df["Mouse ID"]=="s185")]
# s185_df 
# Series for time points
s185_timepoints = s185_df["Timepoint"]
# Series for tumor volume
s185_tumorvol = s185_df["Tumor Volume (mm3)"]
# Plot
fig1, ax1 = plt.subplots()
plt.plot(s185_timepoints,s185_tumorvol)
ax1.set_title("Tumor volume over time (s185, Capomulin)")
ax1.set_ylabel("Tumor volume (mm3)")
ax1.set_xlabel("Time point")
plt.show()

In [ ]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen
# Use data frame created earlier:
# capomulin_df = scc_cleaned_df.loc[(scc_cleaned_df["Drug Regimen"]=="Capomulin")]
# Series for mouse weight
weight = capomulin_df.groupby(["Mouse ID"],as_index=False).agg({"Weight (g)":pd.Series.max})
# Series for average tumor volume
ave_tumor_vol = capomulin_df.groupby(["Mouse ID"],as_index=False).agg({"Tumor Volume (mm3)":"mean"})
# Plot tumor volumn versus mouse weight
fig1, ax1 = plt.subplots()
x_values = weight["Weight (g)"]
y_values = ave_tumor_vol["Tumor Volume (mm3)"]
plt.scatter(x_values,y_values)
ax1.set_title("Tumor volume versus mouse weight (Capomulin)")
ax1.set_ylabel("Average tumor volume (mm3)")
ax1.set_xlabel("Weight (g)")
plt.show()

## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen

# Add the linear regression equation and line to plot
fig1, ax1 = plt.subplots()
x_values = weight["Weight (g)"]
y_values = ave_tumor_vol["Tumor Volume (mm3)"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2)) # Plot the linear regression line
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-",label='y={:.2f}x+{:.2f}'.format(slope,intercept))
plt.legend(fontsize=9)
ax1.set_title("Tumor volume versus mouse weight (Capomulin)")
ax1.set_ylabel("Average tumor volume (mm3)")
ax1.set_xlabel("Weight (g)")
print(f"The r-squared is: {rvalue**2}") # Print to correlation ceoefficient
plt.show()